In [8]:
# Load Libraries
import os
import pandas as pd
import numpy as np
import seaborn as sns



PATH = os.getcwd()[:-13]
DATA_PATH = PATH + "Dataset/superstor.csv"

df = pd.read_csv(DATA_PATH)

c:\Users\ecsan\Repositories\Sales-Prediction\
